<a href="https://www.kaggle.com/code/cocoyachi/lightgbm-futuresales?scriptVersionId=112953699" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore')

# DATA PATH
data_path = '../input/competitive-data-science-predict-future-sales/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

> I studied by 'MUST HAVE' book by Shin. I referred to the book.

# 1.1 Feature Engineering 1 : Feature name -> Korean

In [2]:
sales_train = sales_train.rename(columns = {'data':'날짜',
                                            'date_block_num':'월ID',
                                            'shop_id':'상점ID',
                                            'item_id':'상품ID',
                                            'item_price':'판매가',
                                            'item_cnt_day':'판매량'})
sales_train.head()

,date,월ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [3]:
shops = shops.rename(columns={'shop_name':'상점명',
                              'shop_id':'상점ID'})
shops.head()

,상점명,상점ID
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [4]:
items = items.rename(columns={'item_name':'상품명',
                              'item_id':'상품ID',
                              'item_category_id':'상품분류ID'})
items.head()

,상품명,상품ID,상품분류ID
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [5]:
item_categories = item_categories.rename(columns=
                                         {'item_category_name':'상품분류명',
                                          'item_category_id':'상품분류ID'})
item_categories.head()

,상품분류명,상품분류ID
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [6]:
test = test.rename(columns = {'shop_id':'상점ID',
                              'item_id':'상품ID'})
test.head()

,ID,상점ID,상품ID
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


# 1.2 Feature Engineering 2 : Data Downcasting
* Memory optimization and increased training speed

In [7]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast = 'float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print("Compressed {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [8]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

Compressed 33.3%
Compressed 22.1%
Compressed 33.3%
Compressed 22.8%
Compressed 50.0%


# 1.3 Feature Engineering 3 : Data Combination
* We need to predict 'The monthly sales volume of a store by product'

In [9]:
from itertools import product

train = []
# 월ID, 상점ID, 상품ID 조합 생성하기
for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID']==i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID','상점ID','상품ID'] # 기준 피처
# 'train' [list ------> DataFrame]
train = pd.DataFrame(np.vstack(train), columns=idx_features)

train

,월ID,상점ID,상품ID
0,0.0,59.0,22154.0
1,0.0,59.0,2552.0
2,0.0,59.0,2554.0
3,0.0,59.0,2555.0
4,0.0,59.0,2564.0
...,...,...,...
10913845,33.0,21.0,7635.0
10913846,33.0,21.0,7638.0
10913847,33.0,21.0,7640.0
10913848,33.0,21.0,7632.0


# 1.4 Feature Engineering 4 : Add Target(The monthly sales)

In [10]:
# Grouping by idx_features 
group = sales_train.groupby(idx_features).agg({'판매량':'sum'})
# Reset index
group = group.reset_index()
# Feature rename
group = group.rename(columns={'판매량':'월간 판매량'})

group

,월ID,상점ID,상품ID,월간 판매량
0,0.0,0.0,32.0,6.0
1,0.0,0.0,33.0,3.0
2,0.0,0.0,35.0,1.0
3,0.0,0.0,43.0,1.0
4,0.0,0.0,51.0,2.0
...,...,...,...,...
1609119,33.0,59.0,22087.0,6.0
1609120,33.0,59.0,22088.0,2.0
1609121,33.0,59.0,22091.0,1.0
1609122,33.0,59.0,22100.0,1.0


In [11]:
# merge train & group
train = train.merge(group, on=idx_features, how='left')

train

,월ID,상점ID,상품ID,월간 판매량
0,0.0,59.0,22154.0,1.0
1,0.0,59.0,2552.0,NaN
2,0.0,59.0,2554.0,NaN
3,0.0,59.0,2555.0,NaN
4,0.0,59.0,2564.0,NaN
...,...,...,...,...
10913845,33.0,21.0,7635.0,NaN
10913846,33.0,21.0,7638.0,NaN
10913847,33.0,21.0,7640.0,NaN
10913848,33.0,21.0,7632.0,NaN


In [12]:
# Memory Management
import gc # Garbage Collector

del group
gc.collect();

# 1.5 Feature Engineering 5 : concat test

In [13]:
train.tail()

,월ID,상점ID,상품ID,월간 판매량
10913845,33.0,21.0,7635.0,NaN
10913846,33.0,21.0,7638.0,NaN
10913847,33.0,21.0,7640.0,NaN
10913848,33.0,21.0,7632.0,NaN
10913849,33.0,21.0,7440.0,NaN


In [14]:
test['월ID'] = 34

In [15]:
# train & test connecting
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True, # New index (start 0)
                     keys=idx_features  # Criteria feature
                    )

In [16]:
all_data = all_data.fillna(0)
all_data

,월ID,상점ID,상품ID,월간 판매량
0,0.0,59.0,22154.0,1.0
1,0.0,59.0,2552.0,0.0
2,0.0,59.0,2554.0,0.0
3,0.0,59.0,2555.0,0.0
4,0.0,59.0,2564.0,0.0
...,...,...,...,...
11128045,34.0,45.0,18454.0,0.0
11128046,34.0,45.0,16188.0,0.0
11128047,34.0,45.0,15757.0,0.0
11128048,34.0,45.0,19648.0,0.0


# 1.6 Feature Engineering 6 : Make Final Data

In [17]:
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

# Data Downcasting
all_data = downcast(all_data)

# Garbage Collection
del shops, items, item_categories
gc.collect();

Compressed 7.1%


In [18]:
all_data.head()

,월ID,상점ID,상품ID,월간 판매량,상점명,상품명,상품분류ID,상품분류명
0,0.0,59.0,22154.0,1.0,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37.0,Кино - Blu-Ray
1,0.0,59.0,2552.0,0.0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE The House Of Blue Light LP,58.0,Музыка - Винил
2,0.0,59.0,2554.0,0.0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Who Do You Think We Are LP,58.0,Музыка - Винил
3,0.0,59.0,2555.0,0.0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56.0,Музыка - CD фирменного производства
4,0.0,59.0,2564.0,0.0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Perihelion: Live In Concert DVD (К...,59.0,Музыка - Музыкальное видео


In [19]:
all_data = all_data.drop(['상점명','상품명','상품분류명'], axis=1)

# 1.7 Feature Engineering 7 : First Finish
* train data : 2013.01 ~ 2015.09 (ID ~32)
* valid data : 2015.10(ID 33)
* test data : 2015.11(ID 34)

In [20]:
# train data (Feature)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)

# valid data (Feature)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)

# test data (Feature)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# train data (Target)
y_train = all_data[all_data['월ID'] < 33]['월간 판매량']
y_train = y_train.clip(0, 20)  # Target limit 0 ~ 20

# valid data (Target)
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']
y_valid = y_valid.clip(0, 20)  # Target limit 0 ~ 20

In [21]:
del all_data
gc.collect();

# 1.8 Model training and performance verification
* Categorical data with too many eigenvalues in LightGBM should be treated numerically for better performance

In [22]:
import lightgbm as lgb

params = {'metric':'rmse',
          'num_leaves':255,
          'learning_rate':0.01,
          'force_col_wise':True,
          'random_state':10
         }

# Categorical Feature setting
cat_features = ['상점ID','상품분류ID']

# LightGBM train & valid dataset
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

In [23]:
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=500,
                      valid_sets=(dtrain, dvalid),
                      categorical_feature=cat_features,
                      verbose_eval=50)

[LightGBM] [Info] Total Bins 426
[LightGBM] [Info] Number of data points in the train set: 10675678, number of used features: 4
[LightGBM] [Info] Start training from score 0.299125
[50]	training's rmse: 1.14777	valid_1's rmse: 1.06755
[100]	training's rmse: 1.11425	valid_1's rmse: 1.0386
[150]	training's rmse: 1.09673	valid_1's rmse: 1.02671
[200]	training's rmse: 1.08573	valid_1's rmse: 1.02027
[250]	training's rmse: 1.07722	valid_1's rmse: 1.01661
[300]	training's rmse: 1.0698	valid_1's rmse: 1.0138
[350]	training's rmse: 1.06317	valid_1's rmse: 1.01084
[400]	training's rmse: 1.05734	valid_1's rmse: 1.00936
[450]	training's rmse: 1.05224	valid_1's rmse: 1.00818
[500]	training's rmse: 1.04792	valid_1's rmse: 1.00722


if don't want to input parameter 'categorical_feature=cat_features' setting type 'category'
```python
cat_features=['상점ID','상품분류ID']
for cat_feature in cat_features:
    all_data[cat_feature] = all_data[cat_feature].astype('category')
```

In [24]:
preds = lgb_model.predict(X_test).clip(0, 20)

In [25]:
submission['item_cnt_month'] = preds
submission.to_csv('submission.csv', index=False)